# Data Science: Data processing

Typical packages: `pandas`, `plotnine`, `plotly`, `streamlit`

**References**

- [Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/)
- [Python for Data Analysis, 2nd Edition](https://github.com/wesm/pydata-book)

## Introduction to `pandas`

In [ ]:
import numpy as np
import pandas as pd

## Series and Data Frames

### Series objects

A `Series` is like a vector. All elements must have the same type or are nulls.

In [ ]:
s = pd.Series([1,1,2,3] + [None])
s

### Size

In [ ]:
s.size

### Unique Counts

In [ ]:
s.value_counts()

### Special types of series

#### Strings

In [ ]:
words = 'the quick brown fox jumps over the lazy dog'.split()
s1 = pd.Series([' '.join(item) for item in zip(words[:-1], words[1:])])
s1

In [ ]:
s1.str.upper()

In [ ]:
s1.str.split()

In [ ]:
s1.str.split().str[1]

### Categories

In [ ]:
s2 = pd.Series(['Asian', 'Asian', 'White', 'Black', 'White', 'Hispanic'])
s2

In [ ]:
s2 = s2.astype('category')
s2

In [ ]:
s2.cat.categories

In [ ]:
s2.cat.codes

### Dates and times

Datetimes are often useful as indices to a time series.

In [ ]:
import pendulum

In [ ]:
d = pendulum.today()

In [ ]:
d.to_date_string()

In [ ]:
k = 18
s3 = pd.Series(range(k), 
               index=pd.date_range(d.to_date_string(),
                                   periods=k, 
                                   freq='M'))

In [ ]:
s3

In [ ]:
s3['2021']

In [ ]:
s3['2021-01':'2021-06']

If used as a series, then need `dt` accessor method

In [ ]:
s4 = s3.index.to_series()

In [ ]:
s4.dt.day_name()

### DataFrame objects

A `DataFrame` is like a matrix. Columns in a `DataFrame` are `Series`.

- Each column in a DataFrame represents a **variale**
- Each row in a DataFrame represents an **observation**
- Each cell in a DataFrame represents a **value**

In [ ]:
df = pd.DataFrame(dict(num=[1,2,3] + [None]))
df

In [ ]:
df.num

### Index

Row and column identifiers are of `Index` type.

Somewhat confusingly, index is also a a synonym for the row identifiers.

In [ ]:
df.index

#### Setting a column as the row index

In [ ]:
df

In [ ]:
df1 = df.set_index('num')
df1

#### Making an index into a column

In [ ]:
df1.reset_index()

### Columns

This is just a different index object

In [ ]:
df.columns

### Getting raw values

Sometimes you just want a `numpy` array, and not a `pandas` object.

In [ ]:
df.values

## Creating Data Frames

### Manual

In [ ]:
from collections import OrderedDict

In [ ]:
n = 5
dates = pd.date_range(start='now', periods=n, freq='d')
df = pd.DataFrame(OrderedDict(pid=np.random.randint(100, 999, n), 
                              weight=np.random.normal(70, 20, n),
                              height=np.random.normal(170, 15, n),
                              date=dates,
                             ))
df

### From file

You can read in data from many different file types - plain text, JSON, spreadsheets, databases etc. Functions to read in data look like `read_X` where X is the data type.

In [ ]:
%%file measures.txt
pid	weight	height	date
328	72.654347	203.560866	2018-11-11 14:16:18.148411
756	34.027679	189.847316	2018-11-12 14:16:18.148411
185	28.501914	158.646074	2018-11-13 14:16:18.148411
507	17.396343	180.795993	2018-11-14 14:16:18.148411
919	64.724301	173.564725	2018-11-15 14:16:18.148411

In [ ]:
df = pd.read_table('measures.txt')
df

## Indexing Data Frames

### Implicit defaults

if you provide a slice, it is assumed that you are asking for rows.

In [ ]:
df[1:3]

If you provide a singe value or list, it is assumed that you are asking for columns.

In [ ]:
df[['pid', 'weight']]

### Extracting a column

#### Dictionary style access

In [ ]:
df['pid']

#### Property style access

This only works for column names tat are also valid Python identifier (i.e., no spaces or dashes or keywords)

In [ ]:
df.pid

### Indexing by location

This is similar to `numpy` indexing

In [ ]:
df.iloc[1:3, :]

In [ ]:
df.iloc[1:3, 1:4:2]

### Indexing by name

In [ ]:
df.loc[1:3, 'weight':'height']

**Warning**: When using `loc`, the row slice indicates row names, not positions.

In [ ]:
df1 = df.copy()
df1.index = df.index + 1
df1

In [ ]:
df1.loc[1:3, 'weight':'height']

## Structure of a Data Frame

### Data types

In [ ]:
df.dtypes

### Converting data types

#### Using `astype` on one column

In [ ]:
df.pid = df.pid.astype('category')

#### Using `astype` on multiple columns

In [ ]:
df = df.astype(dict(weight=float, height=float))

#### Using a conversion function

In [ ]:
df.date = pd.to_datetime(df.date)

#### Check

In [ ]:
df.dtypes

### Basic properties

In [ ]:
df.size

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
df.info()

### Inspection

In [ ]:
df.head(n=3)

In [ ]:
df.tail(n=3)

In [ ]:
df.sample(n=3)

In [ ]:
df.sample(frac=0.5)

## Selecting, Renaming and Removing Columns

### Selecting columns

In [ ]:
df.filter(items=['pid', 'date'])

In [ ]:
df.filter(regex='.*ght')

#### Note that you can also use regular string methods on the columns

In [ ]:
df.loc[:, df.columns.str.contains('d')]

### Renaming columns

In [ ]:
df.rename(dict(weight='w', height='h'), axis=1)

In [ ]:
orig_cols = df.columns 

In [ ]:
df.columns = list('abcd')

In [ ]:
df

In [ ]:
df.columns = orig_cols

In [ ]:
df

### Removing columns

In [ ]:
df.drop(['pid', 'date'], axis=1)

In [ ]:
df.drop(columns=['pid', 'date'])

In [ ]:
df.drop(columns=df.columns[df.columns.str.contains('d')])

## Selecting, Renaming and Removing Rows

### Selecting rows

In [ ]:
df[df.weight.between(60,70)]

In [ ]:
df[(69 <= df.weight) & (df.weight < 70)]

In [ ]:
df[df.date.between(pd.to_datetime('2018-11-13'), 
                   pd.to_datetime('2018-11-15 23:59:59'))]

### Renaming rows

In [ ]:
df.rename({i:letter for i,letter in enumerate('abcde')})

In [ ]:
df.index = ['the', 'quick', 'brown', 'fox', 'jumphs']

In [ ]:
df

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df

### Dropping rows

In [ ]:
df.drop([1,3], axis=0)

#### Dropping duplicated data

In [ ]:
df['something'] = [1,1,None,2,None]

In [ ]:
df.loc[df.something.duplicated()]

In [ ]:
df.drop_duplicates(subset='something')

#### Dropping missing data

In [ ]:
df

In [ ]:
df.something.fillna(0)

In [ ]:
df.something.ffill()

In [ ]:
df.something.bfill()

In [ ]:
df.something.interpolate()

In [ ]:
df.dropna()

## Transforming and Creating Columns

In [ ]:
df.assign(bmi=df['weight'] / (df['height']/100)**2)

In [ ]:
df['bmi'] = df['weight'] / (df['height']/100)**2

In [ ]:
df

In [ ]:
df['something'] = [2,2,None,None,3]

In [ ]:
df

## Sorting Data Frames

### Sort on indexes

In [ ]:
df.sort_index(axis=1)

In [ ]:
df.sort_index(axis=0, ascending=False)

### Sort on values

In [ ]:
df.sort_values(by=['something', 'bmi'], ascending=[True, False])

## Summarizing

### Apply an aggregation function

In [ ]:
df.select_dtypes(include=np.number)

In [ ]:
df.select_dtypes(include=np.number).agg(np.sum)

In [ ]:
df.agg(['count', np.sum, np.mean])

## Split-Apply-Combine

We often want to perform subgroup analysis (conditioning by some discrete or categorical variable). This is done with `groupby` followed by an aggregate function. Conceptually, we split the data frame into separate groups, apply the aggregate function to each group separately, then combine the aggregated results back into a single data frame.

In [ ]:
df['treatment'] = list('ababa')

In [ ]:
df

In [ ]:
grouped = df.groupby('treatment')

In [ ]:
grouped.get_group('a')

In [ ]:
grouped.mean()

### Using `agg` with `groupby`

In [ ]:
grouped.agg('mean')

In [ ]:
grouped.agg(['mean', 'std'])

In [ ]:
grouped.agg({'weight': ['mean', 'std'], 'height': ['min', 'max'], 'bmi': lambda x: (x**2).sum()})

### Using `trasnform` wtih `groupby`

In [ ]:
g_mean = grouped[['weight', 'height']].transform(np.mean)
g_mean

In [ ]:
g_std = grouped[['weight', 'height']].transform(np.std)
g_std

In [ ]:
(df[['weight', 'height']] - g_mean)/g_std

## Combining Data Frames

In [ ]:
df

In [ ]:
df1 =  df.iloc[3:].copy()

In [ ]:
df1.drop('something', axis=1, inplace=True)
df1

### Adding rows

Note that `pandas` aligns by column indexes automatically.

In [ ]:
df.append(df1, sort=False)

In [ ]:
pd.concat([df, df1], sort=False)

### Adding columns

In [ ]:
df.pid

In [ ]:
df2 = pd.DataFrame(OrderedDict(pid=[649, 533, 400, 600], age=[23,34,45,56]))

In [ ]:
df2.pid

In [ ]:
df.pid = df.pid.astype('int')

In [ ]:
pd.merge(df, df2, on='pid', how='inner')

In [ ]:
pd.merge(df, df2, on='pid', how='left')

In [ ]:
pd.merge(df, df2, on='pid', how='right')

In [ ]:
pd.merge(df, df2, on='pid', how='outer')

### Merging on the index

In [ ]:
df1 = pd.DataFrame(dict(x=[1,2,3]), index=list('abc'))
df2 = pd.DataFrame(dict(y=[4,5,6]), index=list('abc'))
df3 = pd.DataFrame(dict(z=[7,8,9]), index=list('abc'))

In [ ]:
df1

In [ ]:
df2

In [ ]:
df3

In [ ]:
df1.join([df2, df3])

## Fixing common DataFrame issues

### Multiple variables in a column

In [ ]:
df = pd.DataFrame(dict(pid_treat = ['A-1', 'B-2', 'C-1', 'D-2']))
df

In [ ]:
df.pid_treat.str.split('-')

In [ ]:
df.pid_treat.str.split('-').apply(pd.Series, index=['pid', 'treat'])

### Multiple values in a cell

In [ ]:
df = pd.DataFrame(dict(pid=['a', 'b', 'c'], vals = [(1,2,3), (4,5,6), (7,8,9)]))
df

In [ ]:
df[['t1', 't2', 't3']]  = df.vals.apply(pd.Series)
df

In [ ]:
df.drop('vals', axis=1, inplace=True)

In [ ]:
pd.melt(df, id_vars='pid', value_name='vals').drop('variable', axis=1)

## Reshaping Data Frames

Sometimes we need to make rows into columns or vice versa.

### Converting multiple columns into a single column

This is often useful if you need to condition on some variable.

In [ ]:
url = 'https://raw.githubusercontent.com/uiuc-cse/data-fa14/gh-pages/data/iris.csv'
iris = pd.read_csv(url)

In [ ]:
iris.head()

In [ ]:
iris.shape

In [ ]:
df_iris = pd.melt(iris, id_vars='species')

In [ ]:
df_iris.sample(10)

## Pivoting

Sometimes we need to convert categorical values in a column into separate columns. This is often done at the same time as performing a summary.

In [ ]:
df_iris.pivot_table(index='variable', columns='species', values='value', aggfunc='mean')

## Functional style - `apply`, `applymap` and `map`

`apply` can be used to apply a custom function

In [ ]:
scores = pd.DataFrame(
    np.around(np.clip(np.random.normal(90, 10, (5,3)), 0, 100), 1),
    columns = ['math', 'stat', 'biol'],
    index = ['anne', 'bob', 'charles', 'dirk', 'edgar']
)

In [ ]:
scores

In [ ]:
def convert_grade_1(score):
    return np.where(score > 90, 'A', 
                    np.where(score > 80, 'B',
                            np.where(score > 70, 'C', 'F')))

In [ ]:
scores.apply(convert_grade_1)

The `np.where` is a little clumsy - here is an alternative.

In [ ]:
def convert_grade_2(score):
    if score.name == 'math': # math professors are mean
        return np.choose(
            pd.cut(score, [-1, 80, 90, 95, 100], labels=False),
            ['F', 'C', 'B', 'A']
        )    
    else:
        return np.choose(
            pd.cut(score, [-1, 70, 80, 90, 100], labels=False),
            ['F', 'C', 'B', 'A']
        )

In [ ]:
scores.apply(convert_grade_2)

`apply` can be used to avoid explicit looping

In [ ]:
def likely_profession(row):
    if (row.biol > row.math) and (row.biol > row.stat):
        return 'farmer'
    elif (row.math > row.biol) and (row.math > row.stat):
        return 'high school teacher'
    elif (row.stat > row.math) and (row.stat > row.biol):
        return 'actuary'
    else:
        return 'doctor'

In [ ]:
scores.apply(likely_profession, axis=1)

If all else fails, you can loop over `pandas` data frames.

- Be prepared for pitying looks from more snobbish Python coders

Loops are frowned upon because they are not efficient, but sometimes pragmatism beats elegance.

In [ ]:
for idx, row in scores.iterrows():
    print(f'\nidx = {idx}\nrow = {row.index}: {row.values}\n', 
          end='-'*30)

`apply` can be used for reductions along margins

In [ ]:
df = pd.DataFrame(np.random.randint(0, 10, (4,5)), columns=list('abcde'), index=list('wxyz'))

In [ ]:
df

In [ ]:
df.apply(sum, axis=0)

In [ ]:
df.apply(sum, axis=1)

#### For element-wise mapping operations

In [ ]:
import string

In [ ]:
char_map = {i: c for i,c in enumerate(string.ascii_uppercase)}

In [ ]:
df.applymap(lambda x: char_map[x])

#### For mapping a series

In [ ]:
df.assign(b_map = df.b.map(char_map))

## Chaining commands

Sometimes you see this functional style of method chaining that avoids the need for temporary intermediate variables.

In [ ]:
(
    iris.
    sample(frac=0.2).
    filter(regex='s.*').
    assign(both=iris.sepal_length + iris.petal_length).
    query('both > 2').
    groupby('species').agg(['mean', 'sum']).
    pipe(lambda x: np.around(x, 1))
)

## Moving between R and Python in Jupyter

In [ ]:
%load_ext rpy2.ipython

In [ ]:
import warnings
warnings.simplefilter('ignore', FutureWarning)

In [ ]:
iris = %R iris

In [ ]:
iris.head()

In [ ]:
iris_py = iris.copy()
iris_py.Species = iris_py.Species.str.upper()

In [ ]:
%%R -i iris_py -o iris_r

iris_r <- iris_py[1:3,]

In [ ]:
iris_r

In [ ]:
! python3 -m pip install --quiet watermark

In [ ]:
%load_ext watermark

In [ ]:
%watermark -v -iv